In [6]:
import math
import cv2
from sklearn import neighbors
import numpy as np
import pandas as pd
import os
import os.path
import requests
import pickle
from PIL import Image, ImageDraw
import face_recognition
from face_recognition.face_recognition_cli import image_files_in_folder
curd = os.getcwd()
try:  
    os.mkdir("{}/known_people".format(curd))
except:
    pass
try:  
    os.mkdir("{}/models".format(curd))
except:
    pass
try:  
    os.mkdir("{}/test".format(curd))
except:
    pass
try:  
    os.mkdir("{}/unknown_pictures".format(curd))
except:
    pass
names=[]

In [2]:
ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg'}

In [3]:

def predict(frame, knn_clf=None, model_path=None, distance_threshold=0.5):
    

    if knn_clf is None and model_path is None:
        raise Exception("Must supply knn classifier either thourgh knn_clf or model_path")

    if knn_clf is None:
        with open(model_path, 'rb') as f:
            knn_clf = pickle.load(f)

    
   # X_img = face_recognition.load_image_file(X_img_path)
    X_img = frame
    X_face_locations = face_recognition.face_locations(X_img)

    if len(X_face_locations) == 0:
        return []

    faces_encodings = face_recognition.face_encodings(X_img, known_face_locations=X_face_locations)

    closest_distances = knn_clf.kneighbors(faces_encodings, n_neighbors=1)
    are_matches = [closest_distances[0][i][0] <= distance_threshold for i in range(len(X_face_locations))]

    return [(pred, loc) if rec else ("unknown", loc) for pred, loc, rec in zip(knn_clf.predict(faces_encodings), X_face_locations, are_matches)]



In [4]:
#my version
def show_prediction_labels_on_image(frame, predictions):
    
    #pil_image = Image.open(img_path).convert("RGB")
    pil_image = Image.fromarray(frame).convert("RGB")
    draw = ImageDraw.Draw(pil_image)

    for name, (top, right, bottom, left) in predictions:
        draw.rectangle(((left, top), (right, bottom)), outline=(0, 0, 255))

        
        name = name.encode("UTF-8")

        text_width, text_height = draw.textsize(name)
        draw.rectangle(((left, bottom - text_height - 10), (right, bottom)), fill=(0, 0, 255), outline=(0, 0, 255))
        draw.text((left + 6, bottom - text_height - 5), name, fill=(255, 255, 255, 255))

    del draw

    #pil_image.show()
    #cv2.imshow("frame",pil_image)
    return np.asarray(pil_image)



### FaceRecog on 10 stills

In [8]:
#Still Image Tester
#curd = os.getcwd()
for image_file in os.listdir("{}/test".format(curd)):
    full_file_path = os.path.join("{}/test".format(curd), image_file)
    if os.path.splitext(full_file_path)[1][1:] in ALLOWED_EXTENSIONS:

        print("Looking for faces in {}".format(image_file))
        frame = cv2.imread(full_file_path,-1)

        predictions = predict(frame, model_path="{}/models/trained_knn_model.clf".format(curd))

        for name, (top, right, bottom, left) in predictions:
            print("- Found {} at ({}, {})".format(name, left, top))
            names.append(name)
        # Display results overlaid on an image
        #show_prediction_labels_on_image(frame, predictions)
        final_img = show_prediction_labels_on_image(frame, predictions)
        cv2.imshow("X",final_img)
        cv2.waitKey(0)    # Press enter every time to switch images and in the last image press q free kernel.
    else:
        continue
cv2.destroyAllWindows()

Looking for faces in leos.jpg
- Found LeoDiCap at (588, 291)
- Found unknown at (959, 291)
- Found unknown at (1478, 171)
- Found unknown at (141, 350)
Looking for faces in Y.jpg
- Found LeoDiCap at (1526, 666)
- Found unknown at (913, 82)
- Found unknown at (855, 626)
- Found unknown at (325, 641)
- Found JasonMamoa at (253, 81)
- Found unknown at (1491, 133)
Looking for faces in eg.jpg
- Found ElonMusk at (573, 106)
- Found unknown at (872, 156)
- Found unknown at (1031, 236)
- Found unknown at (165, 86)
- Found unknown at (404, 216)
Looking for faces in elon.jpg
- Found ElonMusk at (632, 204)
- Found unknown at (134, 348)
- Found unknown at (1382, 455)
- Found unknown at (1097, 455)
- Found unknown at (1419, 112)
Looking for faces in Front-view-of-students-in-a-classroom-looking-forward-with-on-raising-his-hand.jpg
- Found unknown at (633, 236)
- Found unknown at (1210, 226)
- Found unknown at (110, 325)
- Found unknown at (901, 296)
- Found unknown at (489, 489)
- Found unknown at 

### Predict Attendance 

In [10]:
#Attendance System csv maker     
namesD=pd.DataFrame(names, columns=["Names"])
namesD= namesD[namesD.Names!="unknown"]

attendance= pd.DataFrame(namesD.iloc[:,0].value_counts())
attendance.rename(index=str,columns={'Names': 'Count'},inplace=True)
attendance["Present"] =0

attendance["Count"][0] >= 5 
# This is so because we assume that in a class scanerio our web cam will take 10 photos in a span of a class period(50 min.)
# And the people recognised in atleast 5 images will be marked present

for i in range(attendance.shape[0]):
    if(attendance["Count"][i] > 1):
        attendance["Present"][i] =1

print(attendance)
print()
attendance_final=attendance.drop(['Count'],axis=1)
attendance_final
#attendance_final.to_csv('Attendance.csv')

            Count  Present
LeoDiCap        2        1
ElonMusk        2        1
JasonMamoa      2        1



,Present
LeoDiCap,1
ElonMusk,1
JasonMamoa,1


### Face Recognition on WebCam Video Stream(Surveillance)

In [11]:
#Live Video tester
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FPS, 100)
rat, frame = cap.read()

while(True):
    rat, frame = cap.read()
    predictions = predict(frame, model_path="{}/models/trained_knn_model.clf".format(curd))
    for name, (top, right, bottom, left) in predictions:
        print("- Found {} at ({}, {})".format(name, left, top))
        names.append(name)

        # Display results overlaid on an image
    show_img = show_prediction_labels_on_image(frame, predictions)
    cv2.imshow('img',show_img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


cap.release()    
cv2.destroyAllWindows()

- Found unknown at (637, 266)
- Found unknown at (662, 242)
- Found unknown at (655, 263)
- Found unknown at (613, 242)
- Found unknown at (613, 242)
- Found unknown at (637, 217)
- Found unknown at (675, 242)
- Found JasonMamoa at (139, 426)
- Found unknown at (662, 217)
- Found JasonMamoa at (199, 438)
- Found JasonMamoa at (199, 450)
- Found unknown at (637, 242)
- Found JasonMamoa at (187, 438)
- Found unknown at (637, 242)
- Found JasonMamoa at (199, 426)
- Found unknown at (655, 242)
- Found unknown at (353, 329)
- Found unknown at (637, 242)
- Found JasonMamoa at (432, 350)
- Found unknown at (613, 242)
- Found unknown at (432, 350)
- Found unknown at (613, 242)
- Found JasonMamoa at (432, 350)
- Found unknown at (613, 242)
- Found JasonMamoa at (425, 350)
- Found unknown at (613, 242)
- Found JasonMamoa at (425, 350)
- Found unknown at (613, 242)
- Found JasonMamoa at (425, 350)
- Found unknown at (613, 242)
- Found JasonMamoa at (418, 350)
- Found unknown at (613, 242)
- Found

### Face Recognition on PhoneCam Video Stream(Surveillance)

In [ ]:
#Live Video tester
url = "http://192.168.137.184:8080/shot.jpg"

while(True):
    img_resp = requests.get(url)
    img_arr = np.array(bytearray(img_resp.content),dtype = np.uint8)
    frame = cv2.imdecode(img_arr,-1)
    predictions = predict(frame, model_path="{}/models/trained_knn_model.clf".format(curd))
    for name, (top, right, bottom, left) in predictions:
        print("- Found {} at ({}, {})".format(name, left, top))
        names.append(name)

        # Display results overlaid on an image
    show_img = show_prediction_labels_on_image(frame, predictions)
    cv2.imshow('img',show_img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


  
cv2.destroyAllWindows()